In [ ]:
from PYME.LMVis import VisGUI

%gui wx

In [ ]:
pymevis = VisGUI.ipython_pymevisualize()
pipeline = pymevis.pipeline

In [ ]:
import numpy as np
from PYME.IO import tabular

# PARAMETERS to Change
min_dist = 10       #minimum distance between skeleton vertices in determining the vector that points from each vertex
                    #to the closest vertex that's at least this distance away (helps make nice backbone vectors)
max_dist = 50       #maximum distance between skeleton vertices. This helps speed up the KDTree query
max_nn = 200        #maximum # of nearest neighbors before collapsing them into one point
ball_r = 50         #radius used for query_ball_points
#asprat_thresh = 1.5 # the minimum asprect ratio that cluster sigma0/sigma1 must have (std dev of major and intermediate axes)
#en = 1              #experiment number (1-3) or 0 for figure 5 tubule data
clust_min = 3                                           # Clusters with less than this # of points will be excluded
Rtn4_chan = 'chan1'    # channel name for Rtn4 data
rtn_save = 'rtn4_screenshot'

high_filt=100          # Threshold for max distance a Rtn4 point can be from mesh and still be included in analysis
low_filt=-100          # Threshold for min distance a Rtn4 point can be from mesh and still be included in analysis

savedir = 'K:\\4Pi_data\\Oligomer_analysis\\Oligomer_analysis_V2\\Real_data_analysis'
ver = 'V2'

datadir = ["K:\\4Pi_data\Oligomer_analysis\\Oligomer_data\\20210907_data\\Cell02_DBSCAN-2_ROI_all-clumped_DBSCAN-clusters_measureClusters3D.csv",
           "K:\\4Pi_data\\Oligomer_analysis\\Oligomer_data\\20220930_data\\Cell04_ROI1_all-clumped_MeasureCluster3D_DBSCAN.csv",
           "K:\\4Pi_data\\Oligomer_analysis\\Oligomer_data\\20221004_data\\Cell05_ROI4_all-clumped_measureClusters3D.csv"]
c_points = ["K:\\4Pi_data\\Oligomer_analysis\\Oligomer_data\\20210907_data\\Cell02_DBSCAN-2_ROI_all-clumped_DBSCAN-clusters.hdf",
            "K:\\4Pi_data\\Oligomer_analysis\\Oligomer_data\\20220930_data\\Cell04_ROI1_all-clumped_DBSCAN_Clusters.hdf",
            "K:\\4Pi_data\\Oligomer_analysis\\Oligomer_data\\20221004_data\\Cell05_ROI4_all-clumped_DBSCAN_Clusters.hdf"]
mesh_fn = ["K:\\4Pi_data\\Oligomer_analysis\\Oligomer_data\\20210907_data\\Cell02_DBSCAN-2_ROI_shrinkwrap.stl",
               "K:\\4Pi_data\\Oligomer_analysis\\Oligomer_data\\20220930_data\\Cell04_ROI1_shrinkwrap_1.stl",
               "K:\\4Pi_data\\Oligomer_analysis\\Oligomer_data\\20221004_data\\Cell05_ROI4_shrinkwrap.stl"]
skelly = ["K:\\4Pi_data\\Oligomer_analysis\\Oligomer_data\\20210907_data\\Cell02_DBSCAN-2_ROI_skeleton.stl",
          "K:\\4Pi_data\\Oligomer_analysis\\Oligomer_data\\20220930_data\\Cell04_ROI1_skeleton.stl",
          "K:\\4Pi_data\\Oligomer_analysis\\Oligomer_data\\20221004_data\\Cell05_ROI4_skeleton.stl"]
d_name = ['20210907_Cell02','20220930_Cell04','20221004_Cell05']

In [ ]:
import numpy as np
from PYME.IO import tabular

# generate 3D Gaussian
# points = np.random.randn(100,3)*100

#######################################################################################################################
# Define needed functions
#######################################################################################################################

def add_ds_from_Nx3(points, pipeline, pymevis, ds_name='points', color=None, normals=None):
    """
    Quickly add points/normals to the pipeline and pymevis display.
    """
    
    d = {'x': points[:,0], 'y': points[:,1], 'z': points[:,2]}
    
    if color is not None:
        d['c'] = color
        
    if normals is not None:
        d['xn'] = normals[:,0]
        d['yn'] = normals[:,1]
        d['zn'] = normals[:,2]

    # create tabular mappingFilter data source and add it to pymevis
    pipeline.addDataSource(ds_name, tabular.mappingFilter(d))

    # select this data source (optional, but helps support "default behavior")
    pipeline.selectDataSource(ds_name)

    # Add a pointcloud layer that displays data source named 'points' (the one we just added)
    pymevis.add_pointcloud_layer(ds_name=ds_name)
    
    if normals is not None:
        pymevis.glCanvas.layers[-1].display_normals=True
        pymevis.glCanvas.layers[-1].normal_scaling=25.0

def save_snapshot(canvas, file_name=None):
    if True:
        pixel_size=None
        
        if file_name is None:
            file_name = wx.FileSelector('Save current view as', wildcard="PNG files(*.png)|*.png",
                            flags=wx.FD_SAVE | wx.FD_OVERWRITE_PROMPT)
        
        if file_name:
            snap = canvas.getIm(pixel_size, GL_RGB)
            print(snap.dtype, snap.shape, snap.max())
            if snap.ndim == 3:
                img = PIL.Image.fromarray(snap.transpose(1, 0, 2))
                #img = toimage(snap.transpose(1, 0, 2))
            else:
                img = PIL.Image.fromarray(snap.transpose())
                #img = toimage(snap.transpose())
            
            img = img.transpose(PIL.Image.FLIP_TOP_BOTTOM)
            
            if not file_name.endswith('.png'):
                img.save('{}.png'.format(file_name))
            else:
                img.save('{}'.format(file_name))
                
def interior_points_from_sdf(sdf, r_max=1, centre=(0,0,0), dx_min=1, p=0.1, eps=0):
    '''
    Generate points from a signed distance function. Effectively does octree-like subdivision of the function domain to
    assign points on a regular grid, then passes through a Monte-Carlo acceptance function to simulate labelling efficiency
    
    Parameters
    ----------
    sdf : function
        the signed distance function. Should be of the form dist = sdf(pts) where pts is a 3xN ndarray/
    r_max: float
        The maximum radius of the object (from centre)
    centre : 3-tuple / array of float
        The centre of the object
    dx_min : float
        The target side length of a voxel. Effectively a density parameter (density = 1/dx_min^3).
    p : float
        Monte-Carlo acceptance probability.

    Returns
    -------
    
    verts : 3xN ndarray of fluorophore positions

    '''
    dx = 1.2 * r_max
    
    vx, vy, vz = [v.ravel() for v in np.mgrid[-1:2, -1:2, -1:2]]
    
    verts = dx * np.vstack([vx, vy, vz]) + np.array(centre)[:,None]
#     print(verts.shape)
    
    c_offs = np.array(
        [[-1, -1, -1], [1, -1, -1], [1, 1, -1], [-1, 1, -1], [-1, -1, 1], [1, -1, 1], [1, 1, 1], [-1, 1, 1]])
    #test_offs =
    
    while dx > dx_min:
        dx /= 2.0
        #test and discard.
        corners = [verts + dx * c_offs[i][:, None] for i in range(8)]
        corner_dists = [sdf(c) for c in corners] + [sdf(verts), ]
#         print(corner_dists)
        
        #corner_dists = [sdf(verts),]
        
        #contains_points = np.abs(np.sum([np.sign(c) for c in corner_dists], axis=0)) < 9
        # for some reason the sign test doesn't seem to work properly. Use a generous
        # distance based test instead
        contains_points = np.min([c for c in corner_dists], axis=0) < 2 * dx
#         print(dx)
#         print(corner_dists)
        
        verts = verts[:, contains_points]
#         print(verts.shape)
        
        #subdivide
        verts = np.concatenate([verts + dx * c_offs[i][:, None] for i in range(8)], axis=1)
        #print(verts.shape)
    
    # because we use a fairly relaxed / conservative criterea for discarding above (which will
    # effectively match the cell containing the surface AND it's neighbours)
    # we perform a more stringent test on the final set of points.
    corners = [verts + dx * c_offs[i][:, None] for i in range(8)]
    corner_dists = [sdf(c) for c in corners]
    
    sv = sdf(verts)
    contains_points = ((np.sum([np.sign(c) for c in corner_dists], axis=0) == -8) | (sv < dx)) & (sv < eps)
    verts = verts[:, contains_points]
    
    #print(verts.shape)
    
    return verts[:, np.random.rand(verts.shape[1]) < p]

def grad_sdf(pts, sdf, delta=0.1):
    """
    Gradient of the signed distance function, calculated via central differences.
    
    Parameters
    ----------
    pts : np.array
        3 x N array of points on which we evaluate the gradient
    sdf : function
        Signed-distance function. Expects 3xN vector of x, y, z coordinates.
    delta : float
        Shift in gradient direction.
    """
    d2 = delta/2.0
    hx = np.array([d2,0,0])[:,None]
    hy = np.array([0,d2,0])[:,None]
    hz = np.array([0,0,d2])[:,None]
    dx = (sdf(pts + hx) - sdf(pts - hx))/delta
    dy = (sdf(pts + hy) - sdf(pts - hy))/delta
    dz = (sdf(pts + hz) - sdf(pts - hz))/delta
    
    return dx, dy, dz

def fullprint(*args, **kwargs):
  from pprint import pprint
  import numpy
  opt = numpy.get_printoptions()
  numpy.set_printoptions(threshold=numpy.inf)
  pprint(*args, **kwargs)
  numpy.set_printoptions(**opt)
    
def proj_u_onto_plane(u, n):
    ''' 
    Input:
        u: vector to be projected onto plane
        n: vector normal to the plane
    Output:
        u_p: normalized projection of vector u onto plane with normal n
    '''
    n_norm = np.sqrt(sum(n**2))
    proj_of_u_on_n = (np.dot(u, n)/n_norm**2)*n
    u_p = u - proj_of_u_on_n
    return u_p

In [ ]:
# Read data except for axes which are more involved
for dset in range(len(datadir)):
    c_data = np.genfromtxt(datadir[dset], delimiter = ',')
    clust = {"count": c_data[:,0], "x": c_data[:,1], "y": c_data[:,2], "z": c_data[:,3], "gyrationRadius": c_data[:,4], 
            "median_abs_deviation": c_data[:,5], "sigma0": c_data[:,9], "sigma1": c_data[:,10], "sigma2": c_data[:,11], 
            "sigma_x": c_data[:,12], "sigma_y": c_data[:,13], "sigma_z": c_data[:,14], "anisotropy": c_data[:,15], 
            "theta": c_data[:,16], "phi": c_data[:,17]}

    #making save-folder if it doesn't exist (DON'T TOUCH THE NEXT 5 LINES)
    from pathlib import Path
    import os
    p = Path(savedir)
    if not p.exists():
        os.mkdir(savedir)
    workdir = os.getcwd()

    import pandas as pd
    import copy

    if 'check1' in locals():
        del check1
    if 'check2' in locals():
        del check2
    if 'x' in locals():
        del x
    if 'y' in locals():
        del y
    if 'z' in locals():
        del z

    # Initialize variables
    csv_file = pd.read_csv(datadir[dset], sep=',', header=None)
    clusto = {"axis0": np.copy(csv_file[6][1:].values), "axis1": np.copy(csv_file[7][1:].values), 
              "axis2": np.copy(csv_file[8][1:].values)}
    clust['axis0'] = np.zeros((len(clust['count']),3))
    clust['axis1'] = np.zeros((len(clust['count']),3))
    clust['axis2'] = np.zeros((len(clust['count']),3))


    # Convert 'axis0', 'axis1', and 'axis2' values to floats from strings
    # axis0
    for a in range(len(clusto["axis0"])):
        #print('a:',a)
        if a > 0:
            del x,y,z,check1,check2
        if clusto['axis0'][a][2:4] == '[ ': 
            for b in range(1,len(clusto['axis0'][a])):
                if clusto['axis0'][a][b] == ' ' and 'check1' not in locals():
                    check1 = b
                elif clusto['axis0'][a][b] == ' ' and 'check2' not in locals():
                    x = float(clusto['axis0'][a][3:b])
                    check2 = b
                elif clusto['axis0'][a][b] == ' ' and clusto['axis0'][a][b-1] != ' ' and 'check2' in locals() and 'y' not in locals():             
                    y = float(clusto['axis0'][a][check2:b])
                    z = float(clusto['axis0'][a][b:-2])
        elif clusto['axis0'][a][2] == '[' and clusto['axis0'][a][3] != ' ': 
            for b in range(1,len(clusto['axis0'][a])):
                if clusto['axis0'][a][b] == ' ' and 'check1' not in locals():
                    check1 = b
                    x = float(clusto['axis0'][a][3:b])
                elif clusto['axis0'][a][b] == ' ' and clusto['axis0'][a][b-1] != ' ' and 'check2' not in locals() and 'y' not in locals():# 'check2' not in locals():
                    y = float(clusto['axis0'][a][check1:b])
                    z = float(clusto['axis0'][a][b:-2])
                    check2 = b
        clust['axis0'][a] = np.array([x, y, z])
    del check1, check2, x, y, z, a, b

    # axis1
    for c in range(len(clusto["axis1"])):
        if c > 0:
            del x,y,z,check1,check2
        if clusto['axis1'][c][2:4] == '[ ': 
            for d in range(1,len(clusto['axis1'][c])):
                if clusto['axis1'][c][d] == ' ' and 'check1' not in locals():
                    check1 = d
                elif clusto['axis1'][c][d] == ' ' and 'check2' not in locals():
                    x = float(clusto['axis1'][c][3:d])
                    check2 = d
                elif clusto['axis1'][c][d] == ' ' and clusto['axis1'][c][d-1] != ' ' and 'check2' in locals() and 'y' not in locals():             
                    y = float(clusto['axis1'][c][check2:d])
                    z = float(clusto['axis1'][c][d:-2])
        elif clusto['axis1'][c][2] == '[' and clusto['axis1'][c][3] != ' ': 
            for d in range(1,len(clusto['axis1'][c])):
                if clusto['axis1'][c][d] == ' ' and 'check1' not in locals():
                    check1 = d
                    x = float(clusto['axis1'][c][3:d])
                elif clusto['axis1'][c][d] == ' ' and clusto['axis1'][c][d-1] != ' ' and 'check2' not in locals() and 'y' not in locals():# 'check2' not in locals():
                    y = float(clusto['axis1'][c][check1:d])
                    z = float(clusto['axis1'][c][d:-2])
                    check2 = d
        clust['axis1'][c] = np.array([x, y, z])
    del check1, check2, x, y, z, c, d

    # axis2
    for e in range(len(clusto["axis2"])):
        if e > 0:
            del x,y,z,check1,check2
        if clusto['axis2'][e][2:4] == '[ ':
            for f in range(1,len(clusto['axis2'][e])):
                if clusto['axis2'][e][f] == ' ' and 'check1' not in locals():
                    check1 = f
                elif clusto['axis2'][e][f] == ' ' and 'check2' not in locals():
                    x = float(clusto['axis2'][e][3:f])
                    check2 = f
                elif clusto['axis2'][e][f] == ' ' and clusto['axis2'][e][f-1] != ' ' and 'check2' in locals() and 'y' not in locals():             
                    y = float(clusto['axis2'][e][check2:f])
                    z = float(clusto['axis2'][e][f:-2])
        elif clusto['axis2'][e][2] == '[' and clusto['axis2'][e][3] != ' ':
            for f in range(1,len(clusto['axis2'][e])):
                if clusto['axis2'][e][f] == ' ' and 'check1' not in locals():
                    check1 = f
                    x = float(clusto['axis2'][e][3:f])
                elif clusto['axis2'][e][f] == ' ' and clusto['axis2'][e][f-1] != ' ' and 'check2' not in locals() and 'y' not in locals():# 'check2' not in locals():
                    y = float(clusto['axis2'][e][check1:f])
                    z = float(clusto['axis2'][e][f:-2])
                    check2 = f
        clust['axis2'][e] = np.array([x, y, z])

    # Load and display points
    pymevis.OpenFile(c_points[dset])

    # define mesh sdf
    from PYME.experimental.isosurface import distance_to_mesh

    mesh_sdf = lambda pts: distance_to_mesh(pts.T, mesh)

    # Load and display mesh
    from PYME.experimental._triangle_mesh import TriangleMesh
    from PYME.LMVis.layers.mesh import TriangleRenderLayer

    mesh = TriangleMesh.from_stl(mesh_fn[dset])

    mesh_name = pipeline.new_ds_name('surf')
    pipeline.recipe.namespace[mesh_name] = mesh
    layer = TriangleRenderLayer(pipeline, dsname=mesh_name, method='wireframe', cmap = 'SolidCyan')
    pymevis.add_layer(layer)

    # grab the bounding box
    mesh_bbox = pymevis.glCanvas.layers[-1]._bbox

    # Load and display skeleton (this is replacing the skeleton creation within the script approach because
    # it takes so long to make the skeleton. Better to do it once and then load it in for analyses

    from PYME.experimental._triangle_mesh import TriangleMesh
    from PYME.LMVis.layers.mesh import TriangleRenderLayer

    mesh = TriangleMesh.from_stl(skelly[dset])

    mesh_name = pipeline.new_ds_name('skeleton')
    pipeline.recipe.namespace[mesh_name] = mesh
    layer = TriangleRenderLayer(pipeline, dsname=mesh_name, method='wireframe', cmap = 'SolidMagenta')
    pymevis.add_layer(layer)

    # grab the bounding box
    mesh_bbox = pymevis.glCanvas.layers[-1]._bbox

    # Import ExtractTableChannel Recipe
    from PYME.recipes.localisations import ExtractTableChannel
    recipe = pipeline.recipe
    # Extract Rtn4 points as 'Rtn4' DataSource
    recipe.add_modules_and_execute([ExtractTableChannel(recipe, inputName = 'filtered_localizations', outputName = 'Rtn4',
                                                        channel = Rtn4_chan)])

    # Select simulated DataSource
    pipeline.selectDataSource('Rtn4')

    # Calculate distance from simulated points to surface
    from PYME.recipes.surface_fitting import DistanceToMesh
    recipe = pipeline.recipe
    recipe.add_modules_and_execute([DistanceToMesh(recipe, input_mesh = 'surf0', input_points = 'Rtn4', output = 'Rtn4_filt')])

    # Select simulated datasource that now has the distance_to_surf0 values associated with it
    pipeline.selectDataSource('Rtn4_filt')

    # Dictionary with info about what filter to use and what values
    filt_dict_rtn = {'distance_to_surf0': [low_filt,high_filt]}

    # Use PYME FilterTable to filter by distance to mesh (SDF) using low_filt and high_filt variables
    from PYME.recipes.tablefilters import FilterTable
    recipe.add_modules_and_execute([FilterTable(recipe, inputName = 'Rtn4_filt', filters = filt_dict_rtn, 
                                                outputName = 'sdf_filtered_Rtn4')])

    # Select the simulate points that passed the SDF filter above
    pipeline.selectDataSource('sdf_filtered_Rtn4')

    # Extract x,y,z coordinates for simulated points
    x_rtn = pipeline['x']
    y_rtn = pipeline['y']
    z_rtn = pipeline['z']

    # Merge x,y,z coordinates into one variable 'points_rtn'
    points_rtn = np.c_[x_rtn.ravel(),y_rtn.ravel(),z_rtn.ravel()]

    # Import cKDTree function from SciPy and use it on the skeleton
    pipeline.selectDataSource('skeleton0')
    d_skelly = pipeline.selectedDataSource
    orig_skeleton = d_skelly._vertices['position'][d_skelly._vertices['halfedge']!=-1]
    from scipy.spatial import cKDTree
    tree_orig = cKDTree(orig_skeleton)

    # Move each vertex to the average position of all of its neighbors within a 'ball_r' radius
    # Also trim out vertices that have a # of nearest neighbors > than max_nn (these are the q-tip looking spots)
    # This doesn't make the problem areas better, it just cuts them out entirely
    from collections import OrderedDict
    skeleton = orig_skeleton
    bad_inds = []
    a_ind = tree_orig.query_ball_point(skeleton, ball_r)
    for trim in range(len(a_ind)):
        skeleton[trim] = np.mean(skeleton[a_ind[trim]],axis=0)
        if len(a_ind[trim]) > max_nn:
            bad_inds.append(a_ind[trim])

    flat_bad_inds = [element for sublist in bad_inds for element in sublist]
    flat_unique = list(OrderedDict.fromkeys(flat_bad_inds))
    skeleton = np.delete(skeleton, flat_unique, axis=0)

    # Make KDTree based on the mean skeleton
    mean_tree_skeleton = cKDTree(skeleton)

    # Find the closest neighbor that is at least 'min_dist' away from point being queried.
    nneigh_ind = np.zeros([len(skeleton),1])
    nneigh_dist = np.zeros([len(skeleton),1])
    for aa in range(len(skeleton)):
        dist, ind = mean_tree_skeleton.query(skeleton[aa], k=len(skeleton), distance_upper_bound=max_dist*2)
        for ab in range(len(dist)):
            if dist[ab] > min_dist:
                nneigh_ind[aa] = ind[ab]
                nneigh_dist[aa] = dist[ab]
                break

    # Calculate vectors pointing from each point to its nearest neighbor that is at least 'min_dist' away.
    skel_vecs = np.zeros([len(skeleton),3])
    bad_skel = []
    for ac in range(len(skeleton)):
        if nneigh_ind[ac] >= len(skeleton):
            bad_skel.append(ac)
        else:
            vec = skeleton[ac] - skeleton[int(nneigh_ind[ac][0])]
            skel_vecs[int(ac)] = vec
    bad_skel = np.asarray(bad_skel)
    skeleton = np.delete(skeleton, bad_skel, axis=0)
    nneigh_dist = np.delete(nneigh_dist, bad_skel, axis=0)
    nneigh_ind = np.delete(nneigh_ind, bad_skel, axis=0)
    skel_vecs = np.delete(skel_vecs, bad_skel, axis=0)

    del mean_tree_skeleton
    mean_tree_skeleton = cKDTree(skeleton)

    # Normalize
    skel_dist_mag = np.c_[nneigh_dist.ravel(),nneigh_dist.ravel(),nneigh_dist.ravel()]
    vecs_hat = skel_vecs/skel_dist_mag

    # Add those vectors to the skeleton in the GUI
    # add_ds_from_Nx3(skeleton, pipeline, pymevis, ds_name=f'nn-mean-skel-{min_dist}nm', normals=vecs_hat)

    # skel_dist = distance from each Rtn4 point to closest skeleton point; skel_ind = index of the closest point in 'skeleton'
    skel_dist_rtn, skel_ind_rtn = mean_tree_skeleton.query(points_rtn, k=1)

    ############################# Plot angles of centers of clusters to skeleton #############################

    # map centers of clusters to 'centers' as xyz coordinates
    centers = np.zeros([len(clust['x']),3])
    for g in range(len(clust['x'])):
        #centers.append([clust['x'][g],clust['y'][g],clust['z'][g]])
        centers[g] = [clust['x'][g],clust['y'][g],clust['z'][g]]

    # determine closest skeleton point to each cluster center
    skel_dist_cent, skel_ind_cent = mean_tree_skeleton.query(centers, k=1)

    # vector from 1st nearest neighbor to Rtn cluster center
    v_cent = skeleton[skel_ind_cent]-centers

    # make skel_dist an [n, 3] array to match v_cent
    skel_dist_mag_cent = np.c_[skel_dist_cent.ravel(),skel_dist_cent.ravel(),skel_dist_cent.ravel()]

    # normalized/unit vector pointing from skeleton to Rtn cluster center point (divide each part of the vector by magnitidue of vector)
    vp_cent = v_cent/skel_dist_mag_cent #was v_cent_hat

    # Calculate vector rejections to use for angle analysis
    vecs_hat_ind = vecs_hat[skel_ind_cent] #only relevant skeleton vectors
    v_cent = []
    v_cent_hat = []
    for vrej in range(len(vp_cent)):
        v_temp = vp_cent[vrej]-vecs_hat_ind[vrej]*(np.dot(vp_cent[vrej],vecs_hat_ind[vrej])/np.linalg.norm(vecs_hat_ind[vrej]))
        if vrej == 6:
            print(v_temp)
        v_cent.append(v_temp)
        v_cent_hat_temp = v_temp/np.linalg.norm(v_temp)
        v_cent_hat.append(v_cent_hat_temp)
    v_cent = np.asarray(v_cent)
    v_cent_hat = np.asarray(v_cent_hat)

    # now let's construct the frame of reference
    # Z-direction unit vector
    zdir = np.array([0,0,1])

    # pick dir along skeleton axis as first vector in frame
    # skeleton axis vectors of skeleton points that are closest to each Rtn4 cluster center
    frame0_cent = vecs_hat[skel_ind_cent]

    # component of z not in the direction of e0 or portion of Z-dir that is orthogonal to frame0_cent
    frame1_cent = zdir[None,:] - frame0_cent*(frame0_cent*zdir[None,:]).sum(1)[:,None]

    # orthogonal to other two vectors in the frame
    frame2_cent = np.cross(frame0_cent,frame1_cent,axis=1)

    # make them unit vectors (vector divided its magnitude)
    frame1_cent_norm = frame1_cent/np.linalg.norm(frame1_cent,axis=1)[:,None]
    frame2_cent_norm = frame2_cent/np.linalg.norm(frame2_cent,axis=1)[:,None]

    # Determine angles_rtn between z-direction and 
    angles_cent = np.arccos((v_cent_hat*frame1_cent_norm).sum(1))  # this works because frame1_rtn is also normalized (unit length=1)

    # Add layer of center points with normals set as the vectors that point to closest part of the skeleton
    add_ds_from_Nx3(centers, pipeline, pymevis, 'cluster-centers', normals=clust['axis0'])

    # Calculate handedness of angles
    # data_dict[name_list[ii]+'_angles_'+str(jj)] = data_list[ii]['angles'][jj]
    # data_dict[name_list[ii]+'_sim_angles_'+str(jj)] = data_list[ii]['sim_angles'][jj]
    skel_vecs = vecs_hat_ind
    loc_vecs = v_cent_hat
    hand = np.cross(skel_vecs,loc_vecs)
    zhand = hand[:,2]
    hand_bool = np.ones(len(zhand))
    for neg in range(len(zhand)):
        if zhand[neg] >= 0:
            continue
        elif zhand[neg] < 0:
            hand_bool[neg] = hand_bool[neg] * -1

    angles_cent = angles_cent * hand_bool
    del skel_vecs, loc_vecs, hand, zhand, hand_bool

    # Import ExtractTableChannel Recipe
    from PYME.recipes.localisations import ExtractTableChannel
    recipe = pipeline.recipe
    # Extract cluster center point

    # Select simulated DataSource
    pipeline.selectDataSource('cluster-centers')

    # Calculate distance from simulated points to surface
    from PYME.recipes.surface_fitting import DistanceToMesh
    recipe = pipeline.recipe
    recipe.add_modules_and_execute([DistanceToMesh(recipe, input_mesh = 'surf0', input_points = 'cluster-centers',
                                                   output = 'centers_sdf')])

    # Select simulated datasource that now has the distance_to_surf0 values associated with it
    pipeline.selectDataSource('centers_sdf')

    # Dictionary with info about what filter to use and what values
    filt_dict_rtn = {'distance_to_surf0': [low_filt,high_filt]}

    # Use PYME FilterTable to filter by distance to mesh (SDF) using low_filt and high_filt variables
    from PYME.recipes.tablefilters import FilterTable
    recipe.add_modules_and_execute([FilterTable(recipe, inputName = 'centers_sdf', filters = filt_dict_rtn, 
                                                outputName = 'sdf_filtered_centers')])

    # Select the simulate points that passed the SDF filter above
    pipeline.selectDataSource('sdf_filtered_centers')

    # Filter data

    # Determine how far each cluster center is from the membrane surface so we can filter out those
    # that are too far away
    pipeline.selectDataSource('surf0')
    surf_d = pipeline.selectedDataSource
    surf_verts = surf_d._vertices['position'][surf_d._vertices['halfedge']!=-1]
    surf_norms = surf_d._vertices['normal']
    from scipy.spatial import cKDTree
    tree_surf = cKDTree(surf_verts)
    surf_dist, surf_ind = tree_surf.query(centers, k=1)
    skel_cent_dist, skel_cent_ind = mean_tree_skeleton.query(centers, k=1)

    # Create filters to filter out clusters that are too far from the skeleton and that are less than clust_min in size
    sdf_filt = skel_cent_dist < high_filt
    size_filt = clust['count'] >= clust_min
    # create filter of aspect ratio of clusters (determined by standard deviation)
    major = clust['sigma0']
    minor = clust['sigma1']
    #asprat = major/minor
    #asprat_filt = asprat >= asprat_thresh

    #total_filt = sdf_filt * size_filt * asprat_filt
    total_filt = sdf_filt * size_filt

    # Apply filter
    # filter by aspect ratio of clusters (determined by standard deviation)
    frame0_cent_filt = frame0_cent[total_filt]
    frame1_cent_filt = frame1_cent_norm[total_filt]
    frame2_cent_filt = frame2_cent_norm[total_filt]
    clust_filt = clust['axis0'][total_filt]
    centers_filt = centers[total_filt]
    angles_cent_filt = angles_cent[total_filt]
    v_cent_hat_filt = v_cent_hat[total_filt]

    from numpy import pi
    # 4. clusters whose axes are tangent to surface in any direction (360 degrees) and positioned as real data

    # Vectors can probably just point from each vertex to the closest vertex. That should be pretty random
    # and tangent
    # positions = cluster_center_positions
    # vectors = tangent_to_surface_and_any_direction\
    # followed advice from https://stackoverflow.com/questions/71160423/how-to-sample-points-in-3d-in-python-with-origin-and-normal-vector

    # parameters for this simulation
    r = 1 # radius of points sampled in the circle on the plane (probably leave as 1 always)
    v_num = 50 # number of vectors desired (evenly spaced around the circle)

    # equations for making the vectors
    rho = np.linspace(0, 2*np.pi, v_num)
    x = np.cos(rho) * r
    y = np.sin(rho) * r
    z = np.zeros(rho.shape)
    # positions of simulated clusters
    arot_pos = centers_filt
    # establish initial vectors
    arot_surf_dist, arot_surf_ind = tree_surf.query(arot_pos, k=1)
    arot_skel_dist, arot_skel_ind = mean_tree_skeleton.query(arot_pos, k=1)
    arot_skel_vec = arot_pos - skeleton[arot_skel_ind]
    arot_vecs = np.cross(vecs_hat[arot_skel_ind], arot_skel_vec)
    arot_tub = vecs_hat[arot_skel_ind]

    # Create vectors in a circle in the plane perpendicular to the vector that points from cluster center to skeleton
    f_all = []
    for bd in range(len(arot_pos)):
        p1 = arot_pos[bd]
        n = arot_skel_vec[bd] # vector normal to the plane
        #pp = arot_vecs[bd] #vector orthogonal to normal
        nabs = np.absolute(n)
        indices = (-n).argsort()[:3]
        v = np.zeros(3)
        v[indices[1]] = -n[indices[0]]
        v[indices[0]] = n[indices[1]]
        v[indices[2]] = 0
        u = np.cross(n,v)
        #normalize vectors
        v_norm = v/np.linalg.norm(v)
        u_norm = u/np.linalg.norm(u)
        f = []
        for be in range(len(rho)):
            circ_p = p1 + r * v_norm * np.cos(rho[be]) + r * u_norm * np.sin(rho[be])
            circ_vec = p1-circ_p
            f.append(circ_vec/np.linalg.norm(circ_vec))
        f = np.asarray(f)
        f_all.append(f)
    f_all = np.asarray(f_all)

    del u
    # #Add the positions with each of the simulated cluster vectors (the number of which is equal to 'rho'-1)
    # for bf in range(len(rho)-1):
    #     name = 'all-rots-' + str(bf)
    #     add_ds_from_Nx3(arot_pos, pipeline, pymevis, name, normals=f_all[:,bf])

    # Calculate portion of Z-dir that is orthogonal to frame0_cent (z-axis of the tubule accroding to tubule axis)
    arot_z = zdir[None,:] - vecs_hat[arot_skel_ind]*(vecs_hat[arot_skel_ind]*zdir[None,:]).sum(1)[:,None]

    # project simulated cluster vectors onto xy-ish axis of tubules
    proj_arot_all = []
    for zy in range(len(f_all)):
        proj_arot = []
        for zz in range(len(rho)-1):
            # u = f_all[zy][zz]
            # n = arot_z[zy]
            # n_norm = np.sqrt(sum(n**2))
            # proj = u-(np.dot(u,n)/n_norm**2)*n
            # proj_arot.append(proj)
            proj_arot.append(proj_u_onto_plane(f_all[zy][zz], arot_z[zy]))
        proj_arot = np.asarray(proj_arot)
        proj_clust_norm = proj_arot/np.linalg.norm(proj_arot,axis=1)[:,None]
        proj_arot_all.append(proj_clust_norm)
    proj_arot_all = np.asarray(proj_arot_all)



    # Calculate angle between simulated points/vectors and tubule axis/tubule z-axis
    arot_tangs_all = []
    arot_ptangs_all = []
    arot_zangs_all = []
    for bi in range(len(rho)-1):
        arot_tangs = np.zeros(len(arot_pos))
        arot_ptangs = np.zeros(len(arot_pos))
        arot_zangs = np.zeros(len(arot_pos))
        for bg in range(len(arot_pos)):
            # Take arccos of the dot product to get the angle out
            arot_tangs[bg] = np.arccos(np.dot(arot_tub[bg],f_all[:,bi][bg]))
            arot_ptangs[bg] = np.arccos(np.dot(arot_tub[bg],proj_arot_all[:,bi][bg]))
            arot_zangs[bg] = np.arccos(np.dot(arot_z[bg],f_all[:,bi][bg]))
        # remove NaNs that occur for some reason (something about invalid value for arccos but I don't understand why)
        # it shouldn't matter though since there are so many points anyways and they are all just going to be 0 anyways
        # filter out the same ones for z-axis even though that shouldn't have issues, just to be consistent
        arot_zangs = arot_zangs[~np.isnan(arot_tangs)]
        arot_tangs = arot_tangs[~np.isnan(arot_tangs)]
        arot_ptangs = arot_ptangs[~np.isnan(arot_tangs)]
        arot_tangs_all.append(arot_tangs)
        arot_ptangs_all.append(arot_ptangs)
        arot_zangs_all.append(arot_zangs)
    arot_tangs_all = np.asarray(arot_tangs_all)
    flat_arot_tangs = [element for sublist in arot_tangs_all for element in sublist]
    flat_arot_tangs = np.asarray(flat_arot_tangs)

    arot_ptangs_all = np.asarray(arot_ptangs_all)
    flat_arot_ptangs = [element for sublist in arot_ptangs_all for element in sublist]
    flat_arot_ptangs = np.asarray(flat_arot_ptangs)

    arot_zangs_all = np.asarray(arot_zangs_all)
    flat_arot_zangs = [element for sublist in arot_zangs_all for element in sublist]
    flat_arot_zangs = np.asarray(flat_arot_zangs)
    # convert from radians to degrees
    arot_tdeg = flat_arot_tangs * (180/pi)
    arot_ptdeg = flat_arot_ptangs * (180/pi)
    arot_zdeg = flat_arot_zangs * (180/pi)

    from mpl_toolkits.mplot3d import Axes3D
    import matplotlib.pyplot as plt
    fig = plt.figure(figsize=[10,10])
    ax = fig.add_subplot(111, projection = '3d')
    ax.scatter(proj_arot_all[0][:,0],proj_arot_all[0][:,1],proj_arot_all[0][:,2],label='proj')
    ax.scatter(0,0,0,label='zero')
    ax.scatter(1,0,0,label='x')
    ax.scatter(0,1,0,label='y')
    ax.scatter(f_all[0][:,0],f_all[0][:,1],f_all[0][:,2],label='clust')
    ax.legend(loc='upper right', bbox_to_anchor=(.6, .6, 0.5, 0.5))
    #ax.set_xlim([-1,1])
    #ax.set_ylim([-1,1])
    ax.set_zlim([-1,1])

    # list of hex color options that are color-blind friendly
    CB_color_cycle = ['#377eb8', '#ff7f00', '#4daf4a',
                      '#f781bf', '#a65628', '#984ea3',
                      '#999999', '#e41a1c', '#dede00']

    # Create histogram of the angles from Rtn4 cluster centers to nearest skeleton point
    import matplotlib.pyplot as plt
    from numpy import pi
    plt.close(fig=None)

    n_cent, bins_cent, patches_cent = plt.hist(x=angles_cent[total_filt],bins=16, alpha=1, rwidth=0.85, density=True, color=CB_color_cycle[0], 
                                               range=(0,pi), label='Rtn4 Cluster Centers')
    plt.xlabel('Angle between cluster centers and tubule axes')
    plt.ylabel('Normalized # of events')
    plt.legend()
    # cent_save = savedir + '\\angles_of_cluster_centers' + ver + '.png'
    # plt.savefig(cent_save, dpi=600, facecolor='w', edgecolor='w',
    #         orientation='portrait', papertype=None, format=None,
    #         transparent=False, bbox_inches=None, pad_inches=0.1,
    #         frameon=None, metadata=None)

    """
    =======================
    Histogram on polar axis
    =======================

    """
    import numpy as np
    import matplotlib.pyplot as plt
    from numpy import pi

    # Import note:
    # Make mirror of data (currently all angles are output as positive because there's no orientation in regards to which
    # side of the tubule the cluster center point is)

    # angles_cent_neg = -angles_cent_filt + 2*pi
    # angles_cent_all = np.concatenate((angles_cent_filt,angles_cent_neg), axis=0)
    fig = plt.figure(figsize=[12,12])
    ax = plt.subplot(111, projection='polar')
    hist = ax.hist(x=angles_cent_filt,bins=32, alpha=1, rwidth=0.85, density=True, color=CB_color_cycle[0], 
                   label='Rtn4 Cluster Centers')
    ax.set_rlabel_position(0)  # Move radial labels away from plotted line
    #ax.set_theta_zero_location("N")
    #ax.set_rlabel_position(180)  # Move radial labels away from plotted line
    ax.set_theta_zero_location("N")
    plt.xlabel('Normalized # of events')
    plt.legend()
    plt.legend(loc='best', bbox_to_anchor=(1.2, 1.15))
    polar_cent_save = savedir + '\\polar_angles_of_cluster_centers' + d_name[dset] + '_' + ver + '.png'
    plt.savefig(polar_cent_save, dpi=600, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format=None,
            transparent=False, bbox_inches=None, pad_inches=0.1,
            frameon=None, metadata=None)

    # 3D scatter plot of vectors that are used to calculate angles (sanity check)
    from mpl_toolkits.mplot3d import Axes3D
    import matplotlib.pyplot as plt
    CB_color_cycle = ['#377eb8', '#ff7f00', '#4daf4a',
                      '#f781bf', '#a65628', '#984ea3',
                      '#999999', '#e41a1c', '#dede00']

    fig = plt.figure(figsize=[10,10])
    ax = fig.add_subplot(111, projection = '3d')
    # tubule z-axis
    x = frame1_cent_filt[:,0]
    y = frame1_cent_filt[:,1]
    z = frame1_cent_filt[:,2]
    # random uniform vectors along unit circle
    # x2 = con_r_norm[:,0]
    # y2 = con_r_norm[:,1]
    # z2 = con_r_norm[:,2]
    # tubule axis vectors
    x3 = frame0_cent_filt[:,0]
    y3 = frame0_cent_filt[:,1]
    z3 = frame0_cent_filt[:,2]
    # cluster axis vectors
    x4 = clust_filt[:,0]
    y4 = clust_filt[:,1]
    z4 = clust_filt[:,2]
    ax.scatter(x,y,z,c=CB_color_cycle[0],alpha=0.5,label='Tubule z-axis')
    # ax.scatter(x2,y2,z2,c=CB_color_cycle[1],alpha=0.5,label='Random')
    ax.scatter(x3,y3,z3,c=CB_color_cycle[5],alpha=0.5,label='Tubule axis')
    ax.scatter(x4,y4,z4,c=CB_color_cycle[2],alpha=0.5,label='Cluster major axis')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    ax.legend(loc='upper right', bbox_to_anchor=(.6, .6, 0.5, 0.5))
    scatter3d_save = savedir + '\\3D_scatter_vectors' + d_name[dset] + '_' + ver + '.png'
    plt.savefig(scatter3d_save, dpi=600, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format=None,
            transparent=False, bbox_inches=None, pad_inches=0.1,
            frameon=None, metadata=None)

    ################### Calculate angle between Rtn4 cluster principle axis and skeleton axis ###################

    # Making a list of random angles as a control
    # Note: one might think that drawing random numbers from a uniform distribution is how you get a uniform 
    # distribution of vectors around a unit sphere, but this will actually result in the vectors pointing to 
    # spots bunched at the poles of the sphere. Instead, a gaussian/normal distribution should be used.
    # See: https://mathworld.wolfram.com/SpherePointPicking.html for a more in-depth explanation

    d_len = len(frame0_cent_filt)
    con_r = np.random.default_rng().normal(size=[d_len, 3])
    con_r_norm = con_r/np.linalg.norm(con_r,axis=1)[:,None]

    # project cluster vectors onto the xy-ish plane of the tubule before calculating the angle between the
    # cluster vector and the tubule axis
    proj_clust = []
    for z in range(len(clust_filt)):

        proj_clust.append(proj_u_onto_plane(clust_filt[z], frame1_cent_filt[z]))

        # u = clust_filt[z]
        # n = frame1_cent_filt[z]
        # n_norm = np.sqrt(sum(n**2))
        # proj_clust.append(u-(np.dot(u,n)/n_norm**2)*n)

        #proj_clust.append((np.dot(clust_filt[z],frame1_cent_filt[z])/(np.linalg.norm(frame1_cent_filt[z]))**2)*frame1_cent_filt[z])
    proj_clust = np.asarray(proj_clust)
    proj_clust_norm = proj_clust/np.linalg.norm(proj_clust,axis=1)[:,None]

    # Calculate dot product one cell at a time
    c_tangs = np.zeros(len(frame0_cent_filt))
    cp_tangs = np.zeros(len(frame0_cent_filt))
    c_dot = np.zeros(len(frame0_cent_filt))
    c_zangs = np.zeros(len(frame0_cent_filt))
    control2 = np.zeros(len(frame0_cent_filt))
    zcon = np.zeros(len(frame0_cent_filt))
    con_r_angs = np.zeros(len(frame0_cent_filt))
    for i in range(len(frame0_cent_filt)):
        # Take arccos of the dot product to get the angle out
        c_tangs[i] = np.arccos(np.dot(frame0_cent_filt[i],clust_filt[i])) # not projected onto tubule xy plane
        cp_tangs[i] = np.arccos(np.dot(frame0_cent_filt[i],proj_clust_norm[i])) # projected onto tubule xy plane
        c_dot[i] = np.dot(frame0_cent_filt[i],clust_filt[i])
        c_zangs[i] = np.arccos(np.dot(frame1_cent_filt[i],clust_filt[i]))
        control2[i] = np.arccos(np.dot(frame2_cent_filt[i],clust_filt[i]))
        zcon[i] = np.arccos(np.dot(zdir, clust_filt[i]))
        con_r_angs[i] = np.arccos(np.dot(con_r_norm[i],clust_filt[i]))

    # convert radians to degrees
    c_tdeg = c_tangs * (180/pi)
    cp_tdeg = cp_tangs * (180/pi)
    c_zdeg = c_zangs * (180/pi)
    con2_deg = control2 * (180/pi)
    zcon_deg = zcon * (180/pi)
    con_r_deg = con_r_angs * (180/pi)
    # Plot the angles as a histogram (pi/2 would mean the cluster principle axis is orthogonal to the skeleton axis)
    # (0 or pi would mean the cluster principle axis is parallel to the skeleton axis)
    import matplotlib.pyplot as plt
    from numpy import pi
    plt.close(fig=None)
    n_clust, bins_clust, patches_clust = plt.hist(x=c_tdeg,bins=24, alpha=0.5, rwidth=0.85, density=True, color='green', 
                                                  range=(0,180), label='Rtn4 clusters')
    #x2 = (np.arange(np.min(bins_clust)+cents, np.max(bins_clust)+diff, diff) - cents)
    #plt.plot(x2,np.sin(x2)/2)
    plt.xlabel('Angle between cluster major axis and tubule axis')
    plt.ylabel('Normalized # of events')
    plt.legend()
    clust_save = savedir + '\\angles_of_cluster-axes-degrees' + d_name[dset] + '_' + ver + '.png'
    # plt.savefig(clust_save, dpi=300, facecolor='w', edgecolor='w',
    #         orientation='portrait', papertype=None, format=None,
    #         transparent=False, bbox_inches=None, pad_inches=0.1,
    #         frameon=None, metadata=None)

    # Save a dictionary of counts, bins, and angles to cluster centers to combine experimental results later
    cluster_output = {'c_tdeg': c_tdeg, 'cp_tdeg': cp_tdeg, 'c_zdeg': c_zdeg, 'arot_tdeg': arot_tdeg, 
                      'arot_zdeg': arot_zdeg, 'center_angles': angles_cent_filt}
    save_output_file = savedir + '\\cluster_output_' + d_name[dset] + '_' + ver + '.npy'
    np.save(save_output_file, cluster_output, allow_pickle=True)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
fig = plt.figure(figsize=[10,10])
ax = fig.add_subplot(111, projection = '3d')
ax.scatter(proj_clust_norm[:,0],proj_clust_norm[:,1],proj_clust_norm[:,2],label='proj')
ax.scatter(0,0,0,label='zero')
ax.scatter(1,0,0,label='x')
ax.scatter(0,1,0,label='y')
ax.scatter(clust_filt[:,0],clust_filt[:,1],clust_filt[:,2],label='clust')
ax.legend(loc='upper right', bbox_to_anchor=(.6, .6, 0.5, 0.5))

#ax.scatter(clust_filt[:,0],clust_filt[:,1],clust_filt[:,2])

In [ ]:
# Control histogram of 
CB_color_cycle = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']

import matplotlib.pyplot as plt
from numpy import pi
plt.close(fig=None)
n_clust, bins_clust, patches_clust = plt.hist(x=c_zdeg,bins=12, alpha=0.25, rwidth=0.85, density=True, color=CB_color_cycle[0], 
                                              range=(0,180), label='Tubule z-axis')
n_clust, bins_clust, patches_clust = plt.hist(x=c_tdeg,bins=12, alpha=0.25, rwidth=0.85, density=True, color=CB_color_cycle[1], 
                                              range=(0,180), label='Tubule axis')
n_clust, bins_clust, patches_clust = plt.hist(x=con2_deg,bins=12, alpha=0.25, rwidth=0.85, density=True, color=CB_color_cycle[2], 
                                              range=(0,180), label='Perpindicular\naxis control')
n_clust, bins_clust, patches_clust = plt.hist(x=zcon_deg,bins=12, alpha=0.25, rwidth=0.85, density=True, color=CB_color_cycle[3], 
                                              range=(0,180), label='True Z-Axis\n control')
plt.xlabel('Angle between axes')
plt.ylabel('Normalized # of events')
plt.legend()
control_save = savedir + '\\control_for_cluster-axes-degrees' + ver + '.png'
# plt.savefig(control_save, dpi=300, facecolor='w', edgecolor='w',
#         orientation='portrait', papertype=None, format=None,
#         transparent=False, bbox_inches=None, pad_inches=0.1,
#         frameon=None, metadata=None)

In [ ]:
"""
=================================================
Violin Plots of Angles between axes WITH CONTROLS
=================================================
used a lot of advice from: 
https://towardsdatascience.com/making-publication-quality-figures-in-python-part-iv-violin-plot-and-dendrogram-ed0bb8b23ddd

"""

from scipy.stats import ranksums
from scipy.stats import mannwhitneyu
from scipy.stats import epps_singleton_2samp
from scipy.stats import ks_2samp
from scipy.stats import wilcoxon
from matplotlib.gridspec import GridSpec


# Statistics
from itertools import combinations
# stats_list = [con_deg, c_deg, parcon_tdeg, parcon_zdeg, tang_tdeg, tang_zdeg, rtang_tdeg, rtang_zdeg,
#              arot_tdeg, arot_zdeg]
# stats_list = [c_tdeg, cp_tdeg, c_zdeg, arot_tdeg, arot_ptdeg, arot_zdeg, asph_tdeg, asph_ptdeg, asph_zdeg,
#              ip_tdeg, ip_ptdeg, ip_zdeg]
stats_list= [c_tdeg, arot_tdeg]
# stats_all = np.zeros(len(stats_list),dtype=list)
# stats_all[0] = ks_2samp(stats_list[0], stats_list[1])
# stats_all[1] = ks_2samp(stats_list[1], stats_list[2])
# stats_all[2] = ks_2samp(stats_list[0], stats_list[2])
combo = list(combinations(range(len(stats_list)),2))
stats_all = np.zeros(len(combo),dtype=list)
for i in range(len(combo)):
    stats_all[i] = ks_2samp(stats_list[combo[i][0]], stats_list[combo[i][1]])

def get_whisker(tmp,dataset):
    whisker = []
    for quantile,data in zip(tmp,dataset):
        data = np.array(data)
        q1 = quantile[0]
        median = quantile[1]
        q3 = quantile[2]
        iqr = q3 - q1
        upper = q3 + 1.5 * iqr
        upper = np.clip(upper,q3,data.max())
        lower = q1 - 1.5 * iqr
        lower = np.clip(lower,data.min(),q1)
        whisker.append((upper,lower))
    return whisker

CB_color_cycle = ['#377eb8', '#ff7f00','#a65628',
                  '#f781bf', '#4daf4a',  '#984ea3',
                  '#999999', '#e41a1c', '#dede00']

colors = ['#377eb8', '#377eb8', '#377eb8',
          '#ff7f00', '#ff7f00', '#ff7f00',
          '#4daf4a', '#4daf4a', '#4daf4a', 
         '#a65628', '#a65628', '#a65628']

def stars(p):
   if p < 0.0001:
       return "****"
   elif (p < 0.001):
       return "***"
   elif (p < 0.01):
       return "**"
   elif (p < 0.05):
       return "*"
   else:
       return "ns"

count = 0
#all_sig = [con_deg,c_deg,con_r_deg]
# xlabels = ['Rtn4-Z', 'Rtn4-T', 'Rand-Par-T', 'Rand-Par-Z', 'Rand-Tan-T', 'Rand-Tan-Z',
#            'Real-Tan-T', 'Real-Tan-Z', 'Real-Any-T', 'Real-Any-Z']
xlabels = ['Rtn4-T', 'Rtn4-Proj-T', 'Rtn4-Z', 'Real-Tan-Any-T', 'Real-Tan-Proj-T', 'Real-Tan-Any-Z', 
           'Real-Sph-T', 'Real-Sph-Proj-T', 'Real-Sph-Z', 'In-Plane-T', 'In-Plane-Proj-T', 'In-Plane-Z']
tmp = [np.percentile(data,[25,50,75]) for data in stats_list]
whisker = get_whisker(tmp,stats_list)
positions = [1,1]#np.linspace(0,len(stats_list)-1,len(stats_list))#
# fig,ax = plt.subplots(figsize=[12,12])
# gs1 = GridSpec(8, 2, hspace=0, wspace=0.1)
# vp = ax.violinplot(dataset=stats_list,positions=positions)
fig = plt.figure(tight_layout=True)
gs = GridSpec(1,8)
ax = fig.add_subplot(gs[0,:])
vp = ax.violinplot(dataset=stats_list,positions=positions)
for pc in vp['bodies']:
    pc.set_facecolor(colors[count])
    pc.set_edgecolor('black')
    count = count + 1
    pc.set_linewidth(2)
    pc.set_alpha(0.2)
vp['cmaxes'].set_color('black')
vp['cmins'].set_color('black')
vp['cbars'].set_color('black')
                      
ax.scatter(positions, [quantile[1] for quantile in tmp],
           marker='o', color='white',s=30,zorder=3)
ax.vlines(positions, [quantile[0] for quantile in tmp],
          [quantile[2] for quantile in tmp],
          color='black',linestyle='-',lw=10)
ax.vlines(positions,
          [bound[0] for bound in whisker],
          [bound[1] for bound in whisker],
          color='black',linestyle='-',lw=2)
plt.xlabel('Principle Axes')
plt.ylabel('Degrees')
plt.xticks(positions, xlabels)#('Tubule z-axis', 'Tubule axis', 'Random control'))
# ax2 = fig.add_subplot(gs[0,:])
# ax2.set_xlim([-1,len(stats_list)+1])
# ax2.set_yticks([])
# ax2.set_xticks([])
#ax2.axis('off')
count = 0
#ann1 = np.linspace(0,len(stats_list)-1,len(stats_list))
# while len(stats_list)/count > 1:
#     ann = np.linspace(0,(len(stats_list)/2)-1,len(stats_list)/2)

    

In [ ]:
#data_list = [c_tdeg, c_zdeg, arot_tdeg, arot_zdeg] # [angles between cluster principle axis and tubule axis, angles between cluster principal axis and z-axis, angle between simulated clusters with principal axis in any possible direction tangent to membrane and the tubule axis, same but with z-axis] 
data_list = [c_tdeg, arot_tdeg]
#data_names = [u'\u2220 \u03C4 Real', u'\u2220 \u03C4 Sim']
deg_range = [0,180]
all_counts = []
all_bins = []
fig = plt.figure(tight_layout=True)
for da in range(len(data_list)):
    count, bins = np.histogram(data_list[da], bins=181, density=True, range=deg_range)
    all_counts.append(count)
    all_bins.append(bins)
    plt.step(bins[:-1], count)
    

In [ ]:
v_cent_hat_filt[ninety_filt]

In [ ]:
ninety_filt = c_tdeg == 90
ninety_centers = centers_filt[ninety_filt]
add_ds_from_Nx3(ninety_centers, pipeline, pymevis, 'ninety-centers', normals=v_cent_hat_filt[ninety_filt])

In [ ]:
CB_color_cycle = ['#377eb8', '#ff7f00','#a65628',
                  '#f781bf', '#4daf4a',  '#984ea3',
                  '#999999', '#e41a1c', '#dede00']
plt.rcParams.update({'font.size': 20})
fig, axs = plt.subplots(2,1, figsize=(12,36))
axs[0].step(all_bins[0][:-1], all_counts[0], color=CB_color_cycle[1], linewidth=3, label='Rtn4-tubule')
axs[0].step(all_bins[2][:-1], all_counts[2], color=CB_color_cycle[6], linewidth=3, label='Sim-tubule')
axs[0].set_ylim([0,0.015])
axs[0].legend(loc='upper right')
axs[1].step(all_bins[1][:-1], all_counts[1], color=CB_color_cycle[0], linewidth=3, label='Rtn4-Z')
axs[1].step(all_bins[3][:-1], all_counts[3], color=CB_color_cycle[6], linewidth=3, label='Sim-Z')
# axs[1].step(0, 0, color=CB_color_cycle[1], linewidth=3, label='Rtn4-tubule')
# axs[1].step(0, 0, color=CB_color_cycle[5], linewidth=3, label='Sim-tubule')
axs[0].set_ylabel('Frequency')
axs[1].set_ylabel('Frequency')
#axs[0].set_xlabel('Angle between cluster axis and tubule-axis')
axs[1].set_xlabel('Angle')# between cluster axis and z-axis')
axs[1].set_ylim([0,0.02])
plt.legend()
step_save = savedir + '\\step_angles' + ver + '.png'
plt.savefig(step_save, dpi=600, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None, metadata=None)

In [ ]:
fig = plt.figure(tight_layout=True)
plt.step(bins[:-1], all_counts[1])#/all_counts[3])
plt.step(bins[:-1], all_counts[3])

In [ ]:
combo[29]

In [ ]:
# annotate plot with stats results
for ann in range(len(stats_list)):
    if ann < len(stats_list)-1:
        plt.annotate('', xy=(positions[ann]+0.02,180), xytext=(positions[ann]+0.98,180),
                     arrowprops=dict(facecolor='black',arrowstyle="-",connectionstyle="bar,fraction=0.2"))
        plt.text(((positions[ann]+positions[ann+1])/2), 210, stars(stats_all[ann][1]),
                 horizontalalignment='center', verticalalignment='center')
    else:
        plt.annotate('', xy=(0+0.02,220), xytext=(1+0.98,220),
                     arrowprops=dict(facecolor='black',arrowstyle="-",connectionstyle="bar,fraction=0.1"))
        plt.text(1, 250, stars(stats_all[ann][1]),
                 horizontalalignment='center', verticalalignment='center')
# del_list = [0, len(stats_list)-1, 
# np.delete(combo,
# for ann2 in range(len(combo)-(len(stats_list)-1)):
#         plt.annotate('', xy=(combo[ann][0]+0.02,180+(20*count2)), xytext=(combo[ann][1]+0.98,180+(20*count2)),
#                      arrowprops=dict(facecolor='black',arrowstyle="-",connectionstyle="bar,fraction=0.2"))
#         count2 = count2 + 1
shape_save = savedir + '\\angles_between_axes' + ver + '.png'
plt.ylim([0, 260])
plt.savefig(shape_save, dpi=600, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None, metadata=None)

In [ ]:
positions

In [ ]:
"""
===================================
Violin Plots of Angles between axes
===================================
used a lot of advice from: 
https://towardsdatascience.com/making-publication-quality-figures-in-python-part-iv-violin-plot-and-dendrogram-ed0bb8b23ddd

"""

from scipy.stats import ranksums
from scipy.stats import mannwhitneyu
from scipy.stats import epps_singleton_2samp
from scipy.stats import ks_2samp
from scipy.stats import wilcoxon

# Statistics
from itertools import combinations
stats_list = [con_deg, c_deg, con_r_deg]
stats_all = np.zeros(len(stats_list),dtype=list)
stats_all[0] = ks_2samp(stats_list[0], stats_list[1])
stats_all[1] = ks_2samp(stats_list[1], stats_list[2])
stats_all[2] = ks_2samp(stats_list[0], stats_list[2])
# combo = list(combinations(range(len(stats_list)),2))
# for i in range(len(combo)):
#     stats_all[i] = ks_2samp(stats_list[combo[i][0]], stats_list[combo[i][1]])



def get_whisker(tmp,dataset):
    whisker = []
    for quantile,data in zip(tmp,dataset):
        data = np.array(data)
        q1 = quantile[0]
        median = quantile[1]
        q3 = quantile[2]
        iqr = q3 - q1
        upper = q3 + 1.5 * iqr
        upper = np.clip(upper,q3,data.max())
        lower = q1 - 1.5 * iqr
        lower = np.clip(lower,data.min(),q1)
        whisker.append((upper,lower))
    return whisker

CB_color_cycle = ['#377eb8', '#ff7f00','#a65628',
                  '#f781bf', '#4daf4a',  '#984ea3',
                  '#999999', '#e41a1c', '#dede00']

def stars(p):
   if p < 0.0001:
       return "****"
   elif (p < 0.001):
       return "***"
   elif (p < 0.01):
       return "**"
   elif (p < 0.05):
       return "*"
   else:
       return "ns"

count = 0
#all_sig = [con_deg,c_deg,con_r_deg]
tmp = [np.percentile(data,[25,50,75]) for data in stats_list]
whisker = get_whisker(tmp,stats_list)
positions = np.linspace(0,len(stats_list)-1,len(stats_list))#[1,2,3,4]
fig,ax = plt.subplots(figsize=[12,12])
vp = ax.violinplot(dataset=stats_list,positions=positions)
for pc in vp['bodies']:
    pc.set_facecolor(CB_color_cycle[count])
    pc.set_edgecolor('black')
    count = count + 1
    pc.set_linewidth(2)
    pc.set_alpha(1)
vp['cmaxes'].set_color('black')
vp['cmins'].set_color('black')
vp['cbars'].set_color('black')
                      
ax.scatter(positions, [quantile[1] for quantile in tmp],
           marker='o', color='white',s=30,zorder=3)
ax.vlines(positions, [quantile[0] for quantile in tmp],
          [quantile[2] for quantile in tmp],
          color='black',linestyle='-',lw=10)
ax.vlines(positions,
          [bound[0] for bound in whisker],
          [bound[1] for bound in whisker],
          color='black',linestyle='-',lw=2)
plt.xlabel('Principle Axes')
plt.ylabel('Degrees')
plt.xticks(positions,('Tubule z-axis', 'Tubule axis', 'Random control'))
# annotate plot with stats results
for ann in range(len(stats_list)):
    if ann < len(stats_list)-1:
        plt.annotate('', xy=(positions[ann]+0.02,180), xytext=(positions[ann]+0.98,180),
                     arrowprops=dict(facecolor='black',arrowstyle="-",connectionstyle="bar,fraction=0.2"))
        plt.text(((positions[ann]+positions[ann+1])/2), 210, stars(stats_all[ann][1]),
                 horizontalalignment='center', verticalalignment='center')
    else:
        plt.annotate('', xy=(0+0.02,220), xytext=(1+0.98,220),
                     arrowprops=dict(facecolor='black',arrowstyle="-",connectionstyle="bar,fraction=0.1"))
        plt.text(1, 250, stars(stats_all[ann][1]),
                 horizontalalignment='center', verticalalignment='center')
# del_list = [0, len(stats_list)-1, 
# np.delete(combo,
# for ann2 in range(len(combo)-(len(stats_list)-1)):
#         plt.annotate('', xy=(combo[ann][0]+0.02,180+(20*count2)), xytext=(combo[ann][1]+0.98,180+(20*count2)),
#                      arrowprops=dict(facecolor='black',arrowstyle="-",connectionstyle="bar,fraction=0.2"))
#         count2 = count2 + 1
shape_save = savedir + '\\angles_between_axes' + ver + '.png'
plt.ylim([0, 260])
# plt.savefig(shape_save, dpi=600, facecolor='w', edgecolor='w',
#         orientation='portrait', papertype=None, format=None,
#         transparent=False, bbox_inches=None, pad_inches=0.1,
#         frameon=None, metadata=None)

In [ ]:
# Plot a 2D histogram of the angles between the cluster major axis and the tubule axis (x) and tubule z-axis(y)
fig = plt.figure(figsize=[10,10])
h, xedges, yedges, image = plt.hist2d(c_tdeg, c_zdeg, bins=24, range=[[0,180],[0,180]], cmap=plt.cm.inferno)#, density=True)
plt.xlabel('Angle between cluster axis and tubule axis')
plt.ylabel('Angle between cluster axis and tubule z-axis')
hist_2d_save = savedir + '\\angles_between_axes_2D_histogram' + ver + '.png'
# plt.savefig(hist_2d_save, dpi=600, facecolor='w', edgecolor='w',
#         orientation='portrait', papertype=None, format=None,
#         transparent=False, bbox_inches=None, pad_inches=0.1,
#         frameon=None, metadata=None)

In [ ]:
plt.scatter(c_tdeg,c_zdeg)

In [ ]:
# Plot a 2D histogram of the cluster center angles to skeleton and angle of cluster axis to skeleton
angles_cent_deg = angles_cent_filt * (180/pi)
fig = plt.figure(figsize=[10,10])
h, xedges, yedges, image = plt.hist2d(cp_tdeg, angles_cent_deg, bins=12, range=[[0,180],[0,180]], cmap=plt.cm.inferno)#, density=True)
plt.xlabel('Angle between cluster axis and tubule axis')
plt.ylabel('Angle between cluster center and skeleton')
hist_2d_save = savedir + '\\angles_between_axes_2D_histogram' + ver + '.png'

In [ ]:
fig = plt.figure(figsize=[10,10])
plt.scatter(c_tdeg, c_zdeg)

In [ ]:
###########################################################################################################################
# Plot the standard deviations of each cluster along each principle axis to get a sense for the shape of the clusters
# if they are spherical, the histograms would all look similar. If elongated, sigma0 should have a bigger range/median
###########################################################################################################################
import matplotlib.pyplot as plt
from numpy import pi
plt.close(fig=None)
sig0_filt = clust['sigma0'][total_filt]
sig1_filt = clust['sigma1'][total_filt]
sig2_filt = clust['sigma2'][total_filt]
max_range = np.around(np.max(clust['sigma0']), decimals=-1)+10
n_sig0, bins_sig0, patches_sig0 = plt.hist(x=sig0_filt,bins=20, alpha=0.5, rwidth=0.85, 
                                           density=True, color=CB_color_cycle[0], label='sigma0', 
                                           range=[0,max_range])
n_sig1, bins_sig1, patches_sig1 = plt.hist(x=sig1_filt,bins=20, alpha=0.5, rwidth=0.85, 
                                           density=True, color=CB_color_cycle[1], label='sigma1', 
                                           range=[0,max_range])
n_sig2, bins_sig2, patches_sig2 = plt.hist(x=sig2_filt,bins=20, alpha=0.5, rwidth=0.85, 
                                           density=True, color=CB_color_cycle[2], label='sigma2', 
                                           range=[0,max_range])
plt.xlabel('Angle')
plt.ylabel('Normalized # of events')
plt.legend()


In [ ]:
###########################################################################################################################
# Plot the standard deviations of each cluster along each principle axis to get a sense for the shape of the clusters
# if they are spherical, the violin plots would all look similar. If elongated, sigma0 should have a bigger range/median
###########################################################################################################################
vi_colors = ['red','blue','orange']
count = 0
all_sig = np.transpose(((sig0_filt),(sig1_filt),(sig2_filt)))
np.shape(all_sig)
all_sig = all_sig.astype(float)
violin_dict = plt.violinplot(all_sig, vert=True, showextrema=False, showmedians=True)
for pc in violin_dict['bodies']:
    pc.set_facecolor(CB_color_cycle[count])
    count = count + 1
plt.xlabel('Principle Axes')
plt.ylabel('Standard Deviation')
plt.xticks((1,2,3),('Sigma0', 'Sigma1', 'Sigma2'))
shape_save = savedir + '\\cluster_shape' + ver + '.png'
plt.savefig(shape_save, dpi=600, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None, metadata=None)

In [ ]:
###########################################################################################################################
# Plot sigmax, sigmay, sigmaz to make sure the data is isotropic (this is kind of a proxy since its still looking at 
# clusters and not beads, but there should be no preference for true x,y,z directions)
###########################################################################################################################
import matplotlib.pyplot as plt
from numpy import pi
plt.close(fig=None)
sigx_filt = clust['sigma_x'][total_filt]
sigy_filt = clust['sigma_y'][total_filt]
sigz_filt = clust['sigma_z'][total_filt]
#max_range_xyz = np.around(np.max(clust['sigma0']), decimals=-1)+10
n_x, bins_x, patches_x = plt.hist(x=sigx_filt,bins=20, alpha=0.5, rwidth=0.85, density=True, color=CB_color_cycle[0],
                                           label='axis0-x', range=[0,70])
n_y, bins_y, patches_y = plt.hist(x=sigy_filt,bins=20, alpha=0.5, rwidth=0.85, density=True, color=CB_color_cycle[1],
                                           label='axis0-y', range=[0,70])
n_z, bins_z, patches_z = plt.hist(x=sigz_filt,bins=20, alpha=0.5, rwidth=0.85, density=True, color=CB_color_cycle[2],
                                           label='axis0-z', range=[0,70])
plt.xlabel('Angle')
plt.ylabel('Normalized # of events')
plt.legend()


In [ ]:
# Determine closest mesh vertex to each skeleton point
dist, ind = tree_surf.query(skeleton, k=1)
ss_vecs = skeleton - surf_verts[ind]
ss_vecs_norm = ss_vecs/np.linalg.norm(ss_vecs,axis=1)[:,None]
ss_long_vecs = np.cross(vecs_hat,ss_vecs_norm)
ss_vecs_norm_filt = ss_vecs_norm[skel_cent_ind]
ss_vecs_norm_filt = ss_vecs_norm_filt[total_filt]
ss_long_vecs_filt = ss_long_vecs[skel_cent_ind]
ss_long_vecs_filt = ss_long_vecs_filt[total_filt]

r1_angs = np.zeros(len(frame0_cent_filt))
r2_angs = np.zeros(len(frame0_cent_filt))
for h in range(len(frame0_cent_filt)):
    # Take arccos of the dot product to get the angle out
    r1_angs[h] = np.arccos(np.dot(ss_long_vecs_filt[h],clust_filt[h]))
    r2_angs[h] = np.arccos(np.dot(ss_vecs_norm_filt[h],clust_filt[h]))

plt.hist2d(r1_angs,r2_angs)

In [ ]:
vi_colors = ['red','green','blue']
count = 0
all_sigxyz = np.transpose(((sigx_filt),(sigy_filt),(sigz_filt)))
np.shape(all_sigxyz)
all_sigxyz = all_sigxyz.astype(float)
violin_dict = plt.violinplot(all_sigxyz, vert=True, showextrema=False, showmedians=True)
for pc in violin_dict['bodies']:
    pc.set_facecolor(CB_color_cycle[count])
    count = count + 1
plt.xlabel('Cartesian Axes')
plt.ylabel('Standard Deviation')
plt.xticks((1,2,3),('Sigmax', 'Sigmay', 'Sigmaz'))
lp_save = savedir + '\\xyz_sigma' + ver + '.png'
plt.savefig(lp_save, dpi=600, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None, metadata=None)

In [ ]:
lp_save

In [ ]:
import matplotlib.pyplot as plt
from numpy import pi
plt.close(fig=None)
n_ani, bins_ani, patches_ani = plt.hist(x=clust['anisotropy'],bins=20, alpha=0.5, rwidth=0.85, density=True, color='purple',
                                           label='anisotropy')

In [ ]:
# # TROUBLESHOOTING STUFF -Testing from here down
# #add_ds_from_Nx3(skeleton_rtn[skel_ind_cent], pipeline, pymevis, 'skeleton-used', normals=frame0_cent)
# # c_dot_ind = c_dot == 0
# # c_dot_zeros = centers[c_dot_ind]
# # sum(c_dot_ind)
# # import pickle
# skel_dict = {'skeleton': skeleton_rtn, 'e0':e0_rtn, 'e1': e1_rtn, 'e2': e2_rtn}
# # skel_save = savedir + '\\skeleton.pickle'
# # file_to_save = open(skel_save,"wb")
# # pickle.dump(skel_save, file_to_save)
# # file_to_save.close()
# type(skel_dict['skeleton'])


# translate so origins of vectors are the same
# newvector = skeleton_origin - myvectors_origin
# translatedvector = myvectors_origin + newvector

# do this instead of translation
# project cluster vector into the plane created by e2 and frame2 and dot it with frame2(1: orthogonal to tube, 0: parallel)

# projection operator: outer product of 2 unit normal vector
# outer product of normalized e2 and frame2 (e2_norm[:,:,None]*frame2_norm[:,None,:])
# 'None' tells it which dimenions to broadcast over
oprod = frame0_cent[:,:,None]*frame2_cent_norm[:,None,:]
# Then do projection matrix (3by3) time the cluster vector (3by1) to get projection
proj = oprod*clust['axis0'][:,:,None]
# Then dot it with e2_norm (0: orthogonal) or frame2_norm (1: orthogonal) (just have them all normalized before dotting)
np.shape(proj)

In [ ]:
###########################################################################################################################
# Messing around with plotting all the clusters on top of each other
###########################################################################################################################

all_data = np.genfromtxt(all_data, delimiter = ',')
all_clust = {"t": all_data[:,1], "nPhotons": all_data[:,5], "x": all_data[:,25], "y": all_data[:,23], 
        "z": all_data[:,9], "clumpIndex": all_data[:,26], "clumpSize": all_data[:,27], "objectID": all_data[:,29], 
        "NEvents": all_data[:,30]}

# Pseudo-code for finding coordinates of all points in each cluster and organizing them by cluster
# Idea 1:
# - basic slow way: loop over data from 0 to number of clusters
# - create boolean for each objectID, filter out points not in that cluster and assign the ones that 
#   are to an array
#
# Idea 2:
# - Loop over each data point (about 30k)
# - Identify objectID for that point
# - Assign the x,y,z coordinates of the point to a list of arrays indexed to that objectID
# - list of arrays will be initialized with length = # of clusters(highest objectID)

# Attempt at Idea 2:
# initialize list that will hold all the coordinates with each index being a unique cluster
id_max = int(np.max(all_clust['objectID']))
c_coords = np.empty(id_max+1, dtype=list)
ind_set = set()
for g in range(len(all_clust['x'])):
    ind = int(all_clust['objectID'][g])
    x = all_clust['x'][g]
    y = all_clust['y'][g]
    z = all_clust['z'][g]
    xyz = [x,y,z]
    if not ind in ind_set:
        c_coords[ind] = xyz
    else:
        c_coords[ind] = np.vstack((c_coords[ind], xyz))
    ind_set.add(ind)
    del ind, x, y, z, xyz

from mpl_toolkits import mplot3d
import numpy as np
import matplotlib.pyplot as plt

centered = np.zeros(id_max, dtype=list)
fig = plt.figure()
ax = plt.axes(projection = '3d')
for h in range(1,id_max):
    if np.shape(c_coords[h]) and np.shape(c_coords[h]) > (3,):
        # calculate genters of clusters and translate all points to be centered around [0,0,0]
        cgx = np.sum(c_coords[h][:,0])/len(c_coords[h])
        cgy = np.sum(c_coords[h][:,1])/len(c_coords[h])
        cgz = np.sum(c_coords[h][:,2])/len(c_coords[h])
        centered[h] = c_coords[h] - [cgx, cgy, cgz]
        ax.scatter3D(centered[h][:,0], centered[h][:,1], centered[h][:,2])
plt.show()

In [ ]:
# incomplete defining a function to do all this
def cluster_analysis(c_measures, points):
    '''
    '''
    c_data = np.genfromtxt(c_measures, delimiter = ',')
    clust = {"count": c_data[:,0], "x": c_data[:,1], "y": c_data[:,2], "z": c_data[:,3], "gyrationRadius": c_data[:,4], 
            "median_abs_deviation": c_data[:,5], "sigma0": c_data[:,9], "sigma1": c_data[:,10], "sigma2": c_data[:,11], 
            "sigma_x": c_data[:,12], "sigma_y": c_data[:,13], "sigma_z": c_data[:,14], "anisotropy": c_data[:,15], 
            "theta": c_data[:,16], "phi": c_data[:,17]}

    #making save-folder if it doesn't exist (DON'T TOUCH THE NEXT 5 LINES)
    from pathlib import Path
    import os
    p = Path(savedir)
    if not p.exists():
        os.mkdir(savedir)
    workdir = os.getcwd()

    import pandas as pd
    import copy

    if 'check1' in locals():
        del check1
    if 'check2' in locals():
        del check2
    if 'x' in locals():
        del x
    if 'y' in locals():
        del y
    if 'z' in locals():
        del z

    # Initialize variables
    csv_file = pd.read_csv(datadir, sep=',', header=None)
    # clusto["axis0"] = np.copy(csv_file[6][1:].values)
    # clusto["axis1"] = np.copy(csv_file[7][1:].values)
    # clusto["axis2"] = np.copy(csv_file[8][1:].values)
    clusto = {"axis0": np.copy(csv_file[6][1:].values), "axis1": np.copy(csv_file[7][1:].values), 
              "axis2": np.copy(csv_file[8][1:].values)}
    clust['axis0'] = np.zeros((len(clust['count']),3))
    clust['axis1'] = np.zeros((len(clust['count']),3))
    clust['axis2'] = np.zeros((len(clust['count']),3))
    # clust = {"axis0": np.copy(csv_file[6][1:].values), "axis1": np.copy(csv_file[7][1:].values),
    #         "axis2": np.copy(csv_file[8][1:].values)}
    # clust = {"count": np.copy(c_data[0][1:].values), "x": np.copy(c_data[1][1:].values), "y": np.copy(c_data[2][1:].values), 
    #         "z": np.copy(c_data[3][1:].values), "gyrationRadius": np.copy(c_data[4][1:].values), 
    #         "median_abs_deviation": np.copy(c_data[5][1:].values), "axis0": np.copy(c_data[6][1:].values), 
    #         "axis1": np.copy(c_data[7][1:].values), "axis2": np.copy(c_data[8][1:].values), 
    #         "sigma0": np.copy(c_data[9][1:].values), "sigma1": np.copy(c_data[10][1:].values), 
    #         "sigma2": np.copy(c_data[11][1:].values), "sigma_x": np.copy(c_data[12][1:].values),
    #         "sigma_y": np.copy(c_data[13][1:].values), "sigma_z": np.copy(c_data[14][1:].values), 
    #         "anisotropy": np.copy(c_data[15][1:].values), "theta": np.copy(c_data[16][1:].values), 
    #         "phi": np.copy(c_data[17][1:].values)}
    #clust2 = copy.deepcopy(clust)

    # Convert 'axis0', 'axis1', and 'axis2' values to floats from strings
    # axis0
    for a in range(len(clusto["axis0"])):
        #print('a:',a)
        if a > 0:
            del x,y,z,check1,check2
        if clusto['axis0'][a][2:4] == '[ ': ##### need a different way of parsing the data
            for b in range(1,len(clusto['axis0'][a])):
                if clusto['axis0'][a][b] == ' ' and 'check1' not in locals():
                    #print('b:',b)
                    check1 = b
                elif clusto['axis0'][a][b] == ' ' and 'check2' not in locals():
                    #print('x - b:',b)
                    x = float(clusto['axis0'][a][3:b])
                    check2 = b
                elif clusto['axis0'][a][b] == ' ' and clusto['axis0'][a][b-1] != ' ' and 'check2' in locals() and 'y' not in locals():             
                    #print('yz - b:',b)
                    y = float(clusto['axis0'][a][check2:b])
                    z = float(clusto['axis0'][a][b:-2])
                    #check2 = b
        elif clusto['axis0'][a][2] == '[' and clusto['axis0'][a][3] != ' ': ####resume here
            for b in range(1,len(clusto['axis0'][a])):
                if clusto['axis0'][a][b] == ' ' and 'check1' not in locals():
                    #print('2x b:',b)
                    check1 = b
                    x = float(clusto['axis0'][a][3:b])
                elif clusto['axis0'][a][b] == ' ' and clusto['axis0'][a][b-1] != ' ' and 'check2' not in locals() and 'y' not in locals():# 'check2' not in locals():
                    #print('2yz - b:',b)
                    y = float(clusto['axis0'][a][check1:b])
                    z = float(clusto['axis0'][a][b:-2])
                    check2 = b
        clust['axis0'][a] = np.array([x, y, z])
    #     clust['axis0'][a][0] = x
    #     clust['axis0'][a][1] = y
    #     clust['axis0'][a][2] = z
    del check1, check2, x, y, z, a, b

    # axis1
    for c in range(len(clusto["axis1"])):
        #print('c:',c)
        if c > 0:
            del x,y,z,check1,check2
        if clusto['axis1'][c][2:4] == '[ ': ##### need a different way of parsing the data
            for d in range(1,len(clusto['axis1'][c])):
                if clusto['axis1'][c][d] == ' ' and 'check1' not in locals():
                    #print('d:',d)
                    check1 = d
                elif clusto['axis1'][c][d] == ' ' and 'check2' not in locals():
                    #print('x - d:',d)
                    x = float(clusto['axis1'][c][3:d])
                    check2 = d
                elif clusto['axis1'][c][d] == ' ' and clusto['axis1'][c][d-1] != ' ' and 'check2' in locals() and 'y' not in locals():             
                    #print('yz - d:',d)
                    y = float(clusto['axis1'][c][check2:d])
                    z = float(clusto['axis1'][c][d:-2])
                    #check2 = d
        elif clusto['axis1'][c][2] == '[' and clusto['axis1'][c][3] != ' ': ####resume here
            for d in range(1,len(clusto['axis1'][c])):
                if clusto['axis1'][c][d] == ' ' and 'check1' not in locals():
                    #print('2x d:',d)
                    check1 = d
                    x = float(clusto['axis1'][c][3:d])
                elif clusto['axis1'][c][d] == ' ' and clusto['axis1'][c][d-1] != ' ' and 'check2' not in locals() and 'y' not in locals():# 'check2' not in locals():
                    #print('2yz - d:',d)
                    y = float(clusto['axis1'][c][check1:d])
                    z = float(clusto['axis1'][c][d:-2])
                    check2 = d
        clust['axis1'][c] = np.array([x, y, z])
    del check1, check2, x, y, z, c, d

    # axis2
    for e in range(len(clusto["axis2"])):
        #print('c:',c)
        if e > 0:
            del x,y,z,check1,check2
        if clusto['axis2'][e][2:4] == '[ ': ##### need a different way of parsing the data
            for f in range(1,len(clusto['axis2'][e])):
                if clusto['axis2'][e][f] == ' ' and 'check1' not in locals():
                    #print('f:',f)
                    check1 = f
                elif clusto['axis2'][e][f] == ' ' and 'check2' not in locals():
                    #print('x - f:',f)
                    x = float(clusto['axis2'][e][3:f])
                    check2 = f
                elif clusto['axis2'][e][f] == ' ' and clusto['axis2'][e][f-1] != ' ' and 'check2' in locals() and 'y' not in locals():             
                    #print('yz - f:',f)
                    y = float(clusto['axis2'][e][check2:f])
                    z = float(clusto['axis2'][e][f:-2])
                    #check2 = f
        elif clusto['axis2'][e][2] == '[' and clusto['axis2'][e][3] != ' ': ####resume here
            for f in range(1,len(clusto['axis2'][e])):
                if clusto['axis2'][e][f] == ' ' and 'check1' not in locals():
                    #print('2x f:',f)
                    check1 = f
                    x = float(clusto['axis2'][e][3:f])
                elif clusto['axis2'][e][f] == ' ' and clusto['axis2'][e][f-1] != ' ' and 'check2' not in locals() and 'y' not in locals():# 'check2' not in locals():
                    #print('2yz - f:',f)
                    y = float(clusto['axis2'][e][check1:f])
                    z = float(clusto['axis2'][e][f:-2])
                    check2 = f
        clust['axis2'][e] = np.array([x, y, z])